In [36]:
import pandas as pd

#en_hi_tweets = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/POS_Hindi_English_Code_Mixed_Tweets.tsv", "r").read()
data2 = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/data2.txt", "r").read()

In [37]:
def token_lookup():
    token_dict = {".":"<PERIOD>", ",": "<COMMA>", ";":"<SEMICOLON>", ":":"<COLON>", "\"":"<QOUTATION>", "'":"QOUTATION", "!":"<EXCLAMATION>", "?":"<QUESTION>", 
                  "(": "<LEFTPARAN>", ")": "<RIGHTPARAN>", "{":"LEFTBRACE", "}":"<RIGHTBRACE>", "[":"<LEFTBRACKET>", "]":"<RIGHTBRACKET>", "--":"<DASH>", "-":"<HYPHEN>",
                  #"\n":"<RETURN>"
                  }
    return token_dict

token_dict = token_lookup()

#for key, token in token_dict.items():
#    en_hi_tweets = en_hi_tweets.replace(key, '{}'.format(token))

sentences = []
#lines = en_hi_tweets.split("\n\t\t\n")
lines = data2.split("\n\n")
for line in lines:
    rows = line.split('\n')
    sent = ['<BOS>']
    for x in rows:
        sent.append("_".join(x.split('\t')))
    sent.append("<EOS>")
    sent = " ".join(sent)
    sentences.append(sent)
print(len(sentences[0]))

172


In [38]:
def create_lookup_tables(sentences):
    vocab = set()
    for line in sentences:
        vocab.update(set(line.split()))
    #vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(sentences)

train_int_text = []

for line in sentences:
    train_int_text.append([vocab_to_int[word] for word in line.split()])

train_int_text = [item for sublist in train_int_text for item in sublist]
print(train_int_text[:1000])  

[1112, 2418, 3633, 3236, 284, 2188, 3446, 1081, 3790, 152, 509, 2285, 2083, 3529, 4078, 3142, 3222, 1112, 617, 3969, 350, 3446, 2744, 1274, 4562, 4747, 3337, 1227, 4228, 296, 1946, 488, 3054, 3222, 1112, 2020, 4530, 1966, 1189, 98, 3976, 2974, 1609, 1524, 4338, 2068, 2017, 1081, 3222, 1112, 3660, 4928, 617, 3969, 350, 3446, 2744, 1274, 4562, 4747, 3337, 1227, 4228, 296, 1946, 488, 3054, 3222, 1112, 225, 1608, 4177, 665, 630, 2328, 2209, 1966, 2991, 3893, 1524, 2509, 1958, 1988, 747, 4023, 2385, 1081, 4981, 418, 1035, 3222, 1112, 2438, 3777, 1578, 1524, 1772, 4346, 1917, 4286, 810, 700, 2265, 2852, 4866, 2406, 1476, 4079, 1081, 7, 1779, 4201, 4604, 3222, 1112, 2776, 4595, 1596, 4982, 1196, 2551, 846, 362, 4187, 3242, 640, 297, 3063, 813, 2776, 221, 1911, 3720, 1183, 289, 4639, 1653, 813, 1414, 2551, 1530, 3376, 948, 3222, 1112, 2114, 2911, 2092, 1540, 4915, 3497, 3927, 1933, 2546, 542, 3694, 2516, 3705, 3796, 4537, 659, 3838, 748, 3222, 1112, 1517, 1397, 1652, 3497, 2547, 3139, 3611, 29

In [42]:
#import helper

#data_dir = './brown.txt'

#helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

from distutils.version import LooseVersion
import warnings
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf2
#import helper
import numpy as np

#int_text, val_int_text, test_int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learn_rate = tf.placeholder(tf.float32, name='learn_rate')
    return (inputs, targets, learn_rate)
    
def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(lstm)#, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    num_layers = 2
    cell = tf2.compat.v1.nn.rnn_cell.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return (cell, initial_state)
    
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)#,initial_state=initial_state)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state
    
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf2.compat.v1.layers.dense(outputs, vocab_size, activation=None)
    return logits, final_state
    
def get_batches(int_text, batch_size, seq_length):
    words_per_batch = batch_size * seq_length
    num_batch = len(int_text)//words_per_batch
    
    input_text = np.array(int_text[:(num_batch*words_per_batch)])
    target_text = np.array(int_text[1:(num_batch * words_per_batch) + 1])

    target_text[-1] = input_text[0]
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    return batches
    
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = '/content/drive/MyDrive/nlp_project/models/model_2_postag/save'

batches = get_batches(train_int_text, batch_size, seq_length)

In [43]:
!pip install tensorflow_addons
import tensorflow_addons as tfa


train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = tfa.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    
#batches = get_batches(train_int_text, batch_size, seq_length)
#train_graph = tf.Graph()
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    
# Save parameters for checkpoint
#helper.save_params((seq_length, save_dir))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/pyth

Epoch   0 Batch    0/12   train_loss = 8.523
Epoch   4 Batch    2/12   train_loss = 6.882
Epoch   8 Batch    4/12   train_loss = 6.668
Epoch  12 Batch    6/12   train_loss = 6.182
Epoch  16 Batch    8/12   train_loss = 5.714
Epoch  20 Batch   10/12   train_loss = 5.390
Epoch  25 Batch    0/12   train_loss = 4.828
Epoch  29 Batch    2/12   train_loss = 4.553
Epoch  33 Batch    4/12   train_loss = 4.260
Epoch  37 Batch    6/12   train_loss = 3.873
Epoch  41 Batch    8/12   train_loss = 3.345
Epoch  45 Batch   10/12   train_loss = 3.094
Epoch  50 Batch    0/12   train_loss = 2.526
Epoch  54 Batch    2/12   train_loss = 2.487
Epoch  58 Batch    4/12   train_loss = 2.020
Epoch  62 Batch    6/12   train_loss = 1.799
Epoch  66 Batch    8/12   train_loss = 1.524
Epoch  70 Batch   10/12   train_loss = 1.324
Epoch  75 Batch    0/12   train_loss = 1.061
Epoch  79 Batch    2/12   train_loss = 0.913
Epoch  83 Batch    4/12   train_loss = 0.719
Epoch  87 Batch    6/12   train_loss = 0.636
Epoch  91 

In [44]:
import tensorflow as tf
import numpy as np
#import helper

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index = np.where(probabilities == np.max(probabilities))[-1][0]

    return int_to_vocab[index]

#_,_,_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = 15, '/content/drive/MyDrive/nlp_project/models/model_2_postag/save'
token_dict = token_lookup()
def get_tensors(loaded_graph):
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor
    
#test_text = [int_to_vocab[word] for word in test_int_text]
#test_text = " ".join(test_text)
#test_sent_text = test_text.split("<period>")

'''
test_sentences = input("Enter sentence:")
token_dict = token_lookup()
for key, token in token_dict.items():
	test_sentences = test_sentences.replace(key, ' {} '.format(token))
test_sentences = test_sentences.lower()
test_sentences = test_sentences.split("<period>")
'''
#all_perp = []

loaded_graph = tf.Graph()
with tf.compat.v1.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.compat.v1.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    #print(final_state)
    # Sentences generation setup
    gen_length = 200
    prime_word = "Chalo_hi_PRT"#"Subah_hi_PART" #'यही' #'moe_szyslak'
    gen_sentences = [prime_word]
            
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
          
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        #pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)

    # Remove tokens
    script = []
    for word in gen_sentences:
        script.append(word.split("_")[0])
    tv_script = ' '.join(script)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/nlp_project/models/model_2_postag/save
Chalo United this guys <EOS> <BOS> I was a total story and I was in IITB way we had a good time to have a good minutes It seemed like the other guy who would have a good time to see who did it was laughing a stamp and it When I see a good minutes who came to the story that I was in the hostel bogs I was going to get noisy so I gave me completely I get into the guy in the lab so strongarm wingmates education we I have a good time to attend so for much the guy in the lab The Bisleri bottle <EOS> <BOS> I am a girl in IITB and just so in my life and mean just broke his heart late do do it do you are no one things are Papa his new points with me of the way with the top english letters in the lab as For a thought He is for sitting on Rs of the secrets is his peace of growing old however our around and used to bug the rest in India <EOS> <BOS> I am a good escort from IITB I have been smokin